In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
# import ydf
import tensorflow_decision_forests as tfdf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs found: {gpus}")
else:
    print("No GPUs found.")

2025-10-06 15:31:54.810311: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759764715.097816      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759764715.187921      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


No GPUs found.


2025-10-06 15:32:16.114892: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [2]:
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
train_df=pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_df=pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [4]:
# preprocess?
def preprocess(df):
    # def cryoSleep_integer(x):
    #     return x.astype(int)
    # df=df.dropna()
    def normalize_name(x):
        x=str(x)
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])
    def split_passenger_id(row):
        if pd.isna(row['PassengerId']):
            return pd.Series([None, None, None])
        arr=row['PassengerId'].split('_')
        if len(arr) < 2:
            arr += [None] * (3 - len(arr))
        
        return pd.Series(arr)
    def split_cabin(row):
        if pd.isna(row['Cabin']):
            return pd.Series([None, None, None])  # Handle None case
        arr = row['Cabin'].split('/')
        # Ensure arr has exactly 3 elements
        if len(arr) < 3:
            arr += [None] * (3 - len(arr))  # Pad with None
        return pd.Series(arr)

    # Apply the function and create new columns
    df["Name"] = df["Name"].apply(normalize_name)
    df[['Deck', 'Num', 'Side']] = df.apply(split_cabin, axis=1)
    df[['Group','Place_In_Group']]=df.apply(split_passenger_id,axis=1)
    df['CryoSleep'] = df['CryoSleep'].apply(lambda x: float(x))
    df['VIP'] = df['VIP'].apply(lambda x: float(x))
    
    return df
    
train_df=preprocess(train_df)
test_df=preprocess(test_df)

In [5]:
train_df.head(5)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Num,Side,Group,Place_In_Group
0,0001_01,Europa,0.0,B/0/P,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,B,0,P,0001,01
1,0002_01,Earth,0.0,F/0/S,TRAPPIST-1e,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,F,0,S,0002,01
2,0003_01,Europa,0.0,A/0/S,TRAPPIST-1e,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,A,0,S,0003,01
3,0003_02,Europa,0.0,A/0/S,TRAPPIST-1e,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,A,0,S,0003,02
4,0004_01,Earth,0.0,F/1/S,TRAPPIST-1e,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,F,1,S,0004,01


In [6]:
input_features=list(train_df.columns)
input_features.remove('Transported')
input_features.remove('PassengerId')
input_features.remove('Cabin')
# imput_features.remove('')

In [7]:
def tokenize_names(features, labels=None):
    features['Name']=tf.strings.split(features['Name'])
    return features, labels

train_ds=tfdf.keras.pd_dataframe_to_tf_dataset(train_df,label="Transported").map(tokenize_names)
test_ds=tfdf.keras.pd_dataframe_to_tf_dataset(test_df).map(tokenize_names)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [8]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0, # Very few logs
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True, # Only use the features in "features"
    random_seed=1234,
)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

I0000 00:00:1759764745.266141      13 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1759764745.267415      13 kernel.cc:783] Collect training examples
I0000 00:00:1759764745.267447      13 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
column_guides {
  column_name_pattern: "^HomePlanet$"
}
column_guides {
  column_name_pattern: "^CryoSleep$"
}
column_guides {
  column_name_pattern: "^Destination$"
}
column_guides {
  column_name_pattern: "^Age$"
}
column_guides {
  column_name_pattern: "^VIP$"
}
column_guides {
  column_name_pattern: "^RoomService$"
}
column_guides {
  column_name_pattern: "^FoodCourt$"
}
column_guides {
  column_name_pattern: "^ShoppingMall$"
}
column_guides {
  column_name_pattern: "^Spa$"
}
column_guides {
  column_name_pattern: "^VRDeck$"
}
column_guides {
  column_name_pattern: "^Name$"
}
column_guides {
  column_name_pattern:

Accuracy: 0.7781456708908081 Loss:0.9171081185340881


In [9]:
model.summary()

Model: "gradient_boosted_trees_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (16):
	Age
	CryoSleep
	Deck
	Destination
	FoodCourt
	Group
	HomePlanet
	Name
	Num
	Place_In_Group
	RoomService
	ShoppingMall
	Side
	Spa
	VIP
	VRDeck

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.      "CryoSleep"  0.324148 ################
    2.           "Name"  0.297003 #############
    3.            "Spa"  0.223101 #####
    4.           "Deck"  0.207020 ###
    5.         "VRDeck"  0.200391 ###
    6.            "Num"  0.195463 ##
    7.    "RoomService"  0.194477 ##
    8.           "Side"  0.188836 #
    9.      "FoodCourt"  0.185526 #
   10.          "Gr

In [10]:
def prediction_to_kaggle_format(model, threshold=0.5):
    proba_survive = model.predict(test_ds, verbose=0)[:,0]
    return pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Transported": (proba_survive >= threshold).astype(str)#.replace({'True': 'True', 'False': 'False'})
    })


def make_submission(kaggle_predictions):
    path="/kaggle/working/submission.csv"
    kaggle_predictions.to_csv(path, index=False)
    print(f"Submission exported to {path}")
    
kaggle_predictions = prediction_to_kaggle_format(model)
make_submission(kaggle_predictions)
!head /kaggle/working/submission.csv

Submission exported to /kaggle/working/submission.csv
PassengerId,Transported
0013_01,True
0018_01,False
0019_01,True
0021_01,True
0023_01,True
0027_01,True
0029_01,True
0032_01,True
0032_02,True


In [11]:
# Basic setup for Gradient Boosted Trees Model
model = tfdf.keras.GradientBoostedTreesModel()
model.fit(train_ds, verbose=0)

tuned_self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {tuned_self_evaluation.accuracy} Loss: {tuned_self_evaluation.loss}")


Use /tmp/tmp16kg1nrr as temporary training directory


I0000 00:00:1759764808.132004      13 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1759764808.132056      13 kernel.cc:783] Collect training examples
I0000 00:00:1759764808.132071      13 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

I0000 00:00:1759764808.132189      13 kernel.cc:401] Number of batches: 9
I0000 00:00:1759764808.132197      13 kernel.cc:402] Number of examples: 8693
I0000 00:00:1759764808.144033      13 data_spec_inference.cc:354] 6466 item(s) have been pruned (i.e. they are considered out of dictionary) for the column Cabin (94 item(s) left) because min_value_count=5 and max_number_of_unique_values=2000
I0000 00:00:17597

Accuracy: 0.7802850604057312 Loss: 0.9340924620628357


In [12]:
predictions = None
num_predictions = 0

for i in range(10):
    print(f"i:{i}")
    # Possible models: GradientBoostedTreesModel or RandomForestModel
    model = tfdf.keras.GradientBoostedTreesModel(
        verbose=0, # Very few logs
        features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
        exclude_non_specified_features=True, # Only use the features in "features"

        min_examples=1,
        categorical_algorithm="RANDOM",
        #max_depth=4,
        shrinkage=0.05,
        #num_candidate_attributes_ratio=0.2,
        split_axis="SPARSE_OBLIQUE",
        sparse_oblique_normalization="MIN_MAX",
        sparse_oblique_num_projections_exponent=2.0,
        num_trees=2000,
        #validation_ratio=0.0,
        random_seed=i,
        honest=True,
    )
    model.fit(train_ds)
    
    sub_predictions = model.predict(test_ds, verbose=0)[:,0]
    if predictions is None:
        predictions = sub_predictions
    else:
        predictions += sub_predictions
    num_predictions += 1

predictions/=num_predictions

kaggle_predictions = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Transported": (predictions >= 0.5).astype(int)
    })

make_submission(kaggle_predictions)

i:0


I0000 00:00:1759764875.805999      13 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1759764875.806052      13 kernel.cc:783] Collect training examples
I0000 00:00:1759764875.806071      13 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
column_guides {
  column_name_pattern: "^HomePlanet$"
}
column_guides {
  column_name_pattern: "^CryoSleep$"
}
column_guides {
  column_name_pattern: "^Destination$"
}
column_guides {
  column_name_pattern: "^Age$"
}
column_guides {
  column_name_pattern: "^VIP$"
}
column_guides {
  column_name_pattern: "^RoomService$"
}
column_guides {
  column_name_pattern: "^FoodCourt$"
}
column_guides {
  column_name_pattern: "^ShoppingMall$"
}
column_guides {
  column_name_pattern: "^Spa$"
}
column_guides {
  column_name_pattern: "^VRDeck$"
}
column_guides {
  column_name_pattern: "^Name$"
}
column_guides {
  column_name_pattern:

i:1


I0000 00:00:1759764993.474608      13 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1759764993.474657      13 kernel.cc:783] Collect training examples
I0000 00:00:1759764993.474670      13 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
column_guides {
  column_name_pattern: "^HomePlanet$"
}
column_guides {
  column_name_pattern: "^CryoSleep$"
}
column_guides {
  column_name_pattern: "^Destination$"
}
column_guides {
  column_name_pattern: "^Age$"
}
column_guides {
  column_name_pattern: "^VIP$"
}
column_guides {
  column_name_pattern: "^RoomService$"
}
column_guides {
  column_name_pattern: "^FoodCourt$"
}
column_guides {
  column_name_pattern: "^ShoppingMall$"
}
column_guides {
  column_name_pattern: "^Spa$"
}
column_guides {
  column_name_pattern: "^VRDeck$"
}
column_guides {
  column_name_pattern: "^Name$"
}
column_guides {
  column_name_pattern:

i:2


...

i:8


I0000 00:00:1759765965.395035      13 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1759765965.395078      13 kernel.cc:783] Collect training examples
I0000 00:00:1759765965.395091      13 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
column_guides {
  column_name_pattern: "^HomePlanet$"
}
column_guides {
  column_name_pattern: "^CryoSleep$"
}
column_guides {
  column_name_pattern: "^Destination$"
}
column_guides {
  column_name_pattern: "^Age$"
}
column_guides {
  column_name_pattern: "^VIP$"
}
column_guides {
  column_name_pattern: "^RoomService$"
}
column_guides {
  column_name_pattern: "^FoodCourt$"
}
column_guides {
  column_name_pattern: "^ShoppingMall$"
}
column_guides {
  column_name_pattern: "^Spa$"
}
column_guides {
  column_name_pattern: "^VRDeck$"
}
column_guides {
  column_name_pattern: "^Name$"
}
column_guides {
  column_name_pattern:

i:9


I0000 00:00:1759766051.932281      13 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1759766051.932328      13 kernel.cc:783] Collect training examples
I0000 00:00:1759766051.932345      13 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
column_guides {
  column_name_pattern: "^HomePlanet$"
}
column_guides {
  column_name_pattern: "^CryoSleep$"
}
column_guides {
  column_name_pattern: "^Destination$"
}
column_guides {
  column_name_pattern: "^Age$"
}
column_guides {
  column_name_pattern: "^VIP$"
}
column_guides {
  column_name_pattern: "^RoomService$"
}
column_guides {
  column_name_pattern: "^FoodCourt$"
}
column_guides {
  column_name_pattern: "^ShoppingMall$"
}
column_guides {
  column_name_pattern: "^Spa$"
}
column_guides {
  column_name_pattern: "^VRDeck$"
}
column_guides {
  column_name_pattern: "^Name$"
}
column_guides {
  column_name_pattern:

Submission exported to /kaggle/working/submission.csv
